# Berekening uitval 

VH verspreidt csv-bestanden (flatfiles) met daarin detail informatie over studiesucces indicatoren. Die zijn handig om te benchmarken.

Hieronder worden ze ingelezen en wordt een benchmark gedaan op uitval.

In [ ]:
import os
import altair as alt


#path = os.path.join(local_data, 'VerenigingHogescholen/2018/gegevensset 2018/flatfiles', 'voortgang_2018.csv')
path = os.path.join(local_data, 'VerenigingHogescholen/2018/gegevensset 2018/flatfiles', 'uitval1_2018.csv')

# inlezen data:
df = pd.read_csv(path
                , encoding='latin1'
                , sep='|')


In [ ]:
# hoe ziet de data eruit?
df.head()

## Filteren

In [ ]:
# maak dataset voor Hogeschool Leiden:
# een variabele met alle G5-brins:
g5_brin = ['21RI','27UM', '22OJ' ,'28DN', '27PZ','25DW']

# maak een aantal filters:
filter_g5 = df.brin_actueel.isin(g5_brin)
filter_bachelor = df.typeho=='ba'
filter_voltijd = df.vorm == 'vt'
filter_hoofdtype = df.inschr_type == '1. hoofdtype'
filter_eerstetype = df.instr_type == "1. eerstetype"

# filter de data zodat je alleen G5-data overhoud.
g5_data = df[(filter_g5) &\
              (filter_bachelor) &\
              (filter_voltijd) &\
              (filter_hoofdtype) &\
              (filter_eerstetype)].copy()

# Groeperen/aggregeren

De data bevat ruwe data, maar nog geen percentages oid per opleiding. Die gaan we hieronder maken.

In [ ]:
groupers = ['inschr_jaar','hogeschool','opleiding'] # <- he Zuke, is dit voor jou eigenlijk wel de bedoelen?

def get_uitval(subdf):
    """Berekent uitval door aantal uitvallers te delen door instroom."""
    uitval_aantal = subdf.uitval.sum()
    totaal_aantal = subdf.instroom.sum()
    uitval = uitval_aantal / totaal_aantal
    return uitval

g5_benchmark = g5_data.groupby(groupers).apply(get_uitval).rename('UitvalNa1Jaar').reset_index()

In [ ]:
# even kijken hoe de data er uit ziet:
g5_benchmark.head()

In [ ]:
# maak er eens een plaatje van:
alt.Chart(g5_benchmark).mark_line().encode(
    x='inschr_jaar'
    , y="UitvalNa1Jaar"
    , color='hogeschool'
)

In [ ]:
# Hmm, hierboven ziet er niet goed uit. Nog een keer:
alt.Chart(g5_benchmark).mark_line().encode(
    x='inschr_jaar'
    , y="UitvalNa1Jaar"
    , color='hogeschool'
).facet(row='opleiding')

# Wat hebben we eigenlijk gemeenschappelijk?

In de lijst met benchmark-plaatjes hierboven zie je dat we veel opleidingen NIET gemeen hebben (dan zie je maar 1 lijn). Hoe zit dat eigenlijk?

In [ ]:
# aantal verschillende opleidingen binnen de G5:
g5_benchmark.opleiding.nunique()

Om het aantal instellingen waar een opleiding aan wordt gegeven, tel je het aantal unieke scholen per opleiding:

In [ ]:
opleiding_aantal_instellingen = g5_benchmark.groupby(['opleiding']).hogeschool.nunique().rename('AantalHogescholen').reset_index()

opleiding_aantal_instellingen

In [ ]:
opleiding_aantal_instellingen.AantalHogescholen.value_counts().sort_index()

Hierboven zie je dus dat er 57 opleidingen zijn, die maar aan 1 hogeschool worden gegeven.

In [ ]:
(131-57)/131

In [ ]:
opleiding_aantal_instellingen[opleiding_aantal_instellingen.AantalHogescholen > 3].opleiding.unique().tolist()

In [ ]:
gemeenschappelijke_opleidingen = opleiding_aantal_instellingen[opleiding_aantal_instellingen.AantalHogescholen > 1].opleiding.unique().tolist()

In [ ]:
g5_benchmark_gem_opl = g5_benchmark[g5_benchmark.opleiding.isin(gemeenschappelijke_opleidingen)]

In [ ]:
# we pimpen het plaatje ook nog wat:
alt.Chart(g5_benchmark_gem_opl).mark_line(point=True).encode(
    x='inschr_jaar'
    , y="UitvalNa1Jaar"
    , color='hogeschool'
    , tooltip=['hogeschool','UitvalNa1Jaar']
).facet(row='opleiding').interactive()